# Interpretability analysis CIFAR10

## Installation of needed libraries

In [ ]:
from IPython.display import clear_output
!pip install build
try:
    import pytorch_trainers_interpretability
except:
    !cd .. && python -m build
    clear_output()
    !pip install --force --upgrade ../dist/pytorch_trainers_interpretability-0.0.1-py3-none-any.whl
    !pip install torchmetrics[image] tqdm
clear_output()
try:
    import pytorch_trainers_interpretability
except:
    raise Exception("Packages not installed! Please try again!")
print("Installation completed!")

## Standard and Adversarial training CIFAR-10

In [1]:
from pytorch_trainers_interpretability.trainers import StandardTrainer, AdversarialTrainer
from pytorch_trainers_interpretability.attack import L2Step
from pytorch_trainers_interpretability.models import ResNet18_cifar
from torchvision import transforms
import matplotlib.pyplot as plt
import torchvision
import torch
import numpy as np

In [ ]:
transform_train = transforms.Compose([
   transforms.RandomCrop(32, padding=4),
  transforms.RandomHorizontalFlip(),
   transforms.ToTensor()])
transform_test = transforms.Compose([
    transforms.ToTensor()])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=True)

normalizer = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
classes = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]


In [ ]:
model = ResNet18_cifar(num_classes=10)
training_kwargs = {
    "model": model,
    "pretrained": False,
    "batch_size": 128,
    "adv_step": L2Step,
    "adv_epsilon": 0.5,
    "adv_iter": 20,
    "adv_lr": 0.1,
    "lr": 0.01,
    "epochs": 100,
    "optimizer": "SGD",
    "weight_decay": 5e-4,
    "lr_scheduler": "CosineAnnealingLR",
    "testset": testset,
    "trainset": trainset,
    "transforms_train": transform_train,
    "transforms_test": transform_test,
    "input_normalizer": normalizer,
    "save_path": "./cifar10_standard_model"
}
trainer = StandardTrainer(**training_kwargs)
trainer()
trainer.eval()
trainer.eval_adv()

In [ ]:
model2 = ResNet18_cifar()
training_kwargs2 = {
    "model": model2,
    "pretrained": False,
    "batch_size": 128,
    "adv_step": L2Step,
    "adv_epsilon": 0.5,
    "adv_iter": 20,
    "adv_lr": 0.1,
    "lr": 0.01,
    "epochs": 100,
    "optimizer": "SGD",
    "weight_decay": 5e-4,
    "lr_scheduler": "CosineAnnealingLR",
    "testset": testset,
    "trainset": trainset,
    "transforms_train": transform_train,
    "transforms_test": transform_test,
    "input_normalizer": normalizer,
    "save_path": "./cifar10_robust_l2_model"
}
trainer2 = AdversarialTrainer(**training_kwargs2)
trainer2()
trainer2.eval()
trainer2.eval_adv()